In [1]:
import tkinter as tk  # Tkinter kütüphanesini içe aktar
import threading  # Çoklu iş parçacığı desteği için
import pyttsx3  # Metni sese dönüştürmek için
import speech_recognition as sr  # Sesli komutları tanımak için
import webbrowser  # Tarayıcıda web sitelerini açmak için
import os  # Sistem komutları için
import datetime  # Tarih ve saat için

In [2]:
# Sesli asistan motorunu başlat
engine = pyttsx3.init()  # pyttsx3 motorunu başlat

In [4]:
def speak(text):
    """Bu fonksiyon metni sese dönüştürüp konuşur."""
    if not engine._inLoop:  # Eğer ses motoru döngüde değilse
        engine.say(text)  # Metni sese çevirir
        engine.runAndWait()  # Sesin çalınmasını bekler

In [6]:
def listen_command():
    """Kullanıcıdan sesli komut alır ve metne dönüştürür."""
    recognizer = sr.Recognizer()  # Tanıyıcıyı oluştur
    with sr.Microphone() as source:  # Mikrofonu kullan
        recognizer.adjust_for_ambient_noise(source)  # Gürültüyü azalt
        print("Listening...")  # Dinliyorum mesajı
        audio = recognizer.listen(source)  # Ses kaydını başlat
        try:
            command = recognizer.recognize_google(audio, language='en-US')  # Ses kaydını metne dönüştür
            print(f"Command: {command}")  # Komutu konsola yazdır
            return command.lower()  # Komutu küçük harfe çevir ve döndür
        except sr.UnknownValueError:  # Anlaşılmayan ses hatası
            speak("Sorry, I didn't catch that. Please repeat.")
            return None

In [9]:
def process_command(command):
    """Bu fonksiyon verilen komuta göre işlem yapar."""
    
    # Merhaba komutu
    if "hello" in command:
        speak("Hello! How can I assist you?")

    # Saat kaç komutu
    elif "what time is it" in command:
        current_time = datetime.datetime.now().strftime('%H:%M')
        speak(f"The current time is {current_time}")

    # Tarih nedir komutu
    elif "what is the date" in command:
        today = datetime.datetime.now().strftime('%Y-%m-%d')
        speak(f"Today's date is {today}")

    # İnterneti aç komutu
    elif "open internet" in command:
        speak("Which website would you like to open?")
        website = listen_command()  # Web sitesi dinleniyor
        if website:
            url = f"https://www.{website}.com"  # URL oluşturuluyor
            webbrowser.open(url)  # Tarayıcıda açılıyor
            speak(f"Opening {website} website.")

    # Bilgisayarı kapat komutu
    elif "shutdown" in command:
        speak("Do you want me to shut down your computer? You can say yes or no.")
        confirmation = listen_command()
        if "yes" in confirmation:
            speak("Shutting down your computer. Goodbye!")
            os.system("shutdown /s /t 1")  # Bilgisayarı kapat
        else:
            speak("Shutdown canceled.")

    # Tanınmayan komut
    else:
        speak("I didn't understand that command. Please say something else.")


In [11]:
def run_assistant():
    """Asistanın ana döngüsünü çalıştırır."""
    while True:
        command = listen_command()  # Komut alınıyor
        if command:
            if "exit" in command or "close" in command:  # Kapatma komutu
                speak("Goodbye!")
                break
            process_command(command)  # Komut işleniyor
            print("Processing...")  # Konsolda işlem durumu yazdırılıyor

In [13]:
def start_assistant():
    """Asistanı ayrı bir iş parçacığında başlatır."""
    threading.Thread(target=run_assistant).start()  # Yeni bir iş parçacığında asistan başlatılıyor

In [15]:
def stop_assistant():
    """Asistanı durdurur."""
    engine.stop()  # Ses motorunu durdur
    speak("Assistant stopped.")  # Durdurma mesajı veriliyor
    root.quit()  # Tkinter uygulaması kapatılıyor

In [17]:
# GUI Kurulumu
root = tk.Tk()  # Tkinter ana pencere oluşturuluyor
root.title("Voice Assistant")  # Pencere başlığı belirleniyor
root.geometry("500x350")  # Pencere boyutu ayarlanıyor

label = tk.Label(root, text="Voice Assistant", font=("Helvetica", 16))  # Başlık etiketi oluşturuluyor
label.pack(pady=20)  # Etiket yerleştiriliyor

In [19]:
# Başlat Butonu
start_button = tk.Button(root, text="Start", command=start_assistant, font=("Helvetica", 12), width=10, height=2)  # Başlat butonu
start_button.pack(pady=10)  # Buton yerleştiriliyor

In [21]:
# Kapat Butonu
stop_button = tk.Button(root, text="Stop", command=stop_assistant, font=("Helvetica", 12), width=10, height=2)  # Kapat butonu
stop_button.pack(pady=10)  # Buton yerleştiriliyor

In [23]:
# Tkinter döngüsünü başlat
root.mainloop()  # Kullanıcı arayüzü döngüsü başlatılıyor